**Exercise:** Whenever you survey people about sensitive issues, you have to deal with [social desirability bias](https://en.wikipedia.org/wiki/Social_desirability_bias), which is the tendency of people to adjust their answers to show themselves in the most positive light.
One way to improve the accuracy of the results is [randomized response](https://en.wikipedia.org/wiki/Randomized_response).

As an example, suppose want to know how many people cheat on their taxes.  
If you ask them directly, it is likely that some of the cheaters will lie.
You can get a more accurate estimate if you ask them indirectly, like this: Ask each person to flip a coin and, without revealing the outcome,

* If they get heads, they report YES.

* If they get tails, they honestly answer the question "Do you cheat on your taxes?"

If someone says YES, we don't know whether they actually cheat on their taxes; they might have flipped yes.
Knowing this, people might be more willing to answer honestly.

Suppose you survey 100 people this way and get 80 YESes and 20 NOs.  Based on this data, what is the posterior distribution for the fraction of people who cheat on their taxes?  What is the most likely quantity in the posterior distribution?

## First approach

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import binom
import seaborn as sns

# build a bayes table and get a distribution with the result of the experiment
# that will act as the prior
hypos = np.linspace(0, 1, 101)  # this represents all possible fractions of people
bt = pd.DataFrame({
    'hypos': hypos
})
positive, trials = 80, 100
bt['likelihood'] = binom.pmf(positive, trials, hypos)
bt['conjuction'] = bt.hypos * bt.likelihood
bt['prior'] = bt.conjuction / bt.conjuction.sum()

# As the probability of getting heads or tails is .5 we need a new likelihood
# that reflects this fact. In our experiment as we have 100 people this will
# mean that 50 of them got tails and the other 50 got heads
bt['flip_likelihood'] = binom.pmf(50, 100, hypos)
bt['flip_conjuction'] = bt.prior * bt.flip_likelihood
bt['posterior'] = bt.flip_conjuction / bt.flip_conjuction.sum()

# Get the MAP
max_prob = bt.posterior.max()
MAP = bt[bt.posterior == max_prob].hypos.values[0]
print(f'The most probable fraction of the people that cheat in their taxes is {MAP}')
print(f'However, the probability of having exactly such fraction is {round(max_prob, 2)}')

# Plot the results
sns.lineplot(x=bt.hypos, y=bt.prior, label='prior');
sns.lineplot(x=bt.hypos, y=bt.posterior, label='posterior');

### Getting 95% of the probability
As a practice I calculated the bounds that account for the 95% of the probability.

In [ ]:
# Get the interval that contains the 95% of the results
# get the CDF
bt['cdf'] = bt.posterior.cumsum()

# get the CDF from .65 and locate the index where it reaches .475
# However we only need have of the probability for the .65 value (as the other
# half belongs to the lower interval)
over_65 = bt.posterior[bt.hypos >= .65]
over_65[65] /= 2 
over_65 = over_65.cumsum()
over_65_index = over_65[over_65 >= .475].index[0]

# now get the distance between .65 index and the .475 index
center_index = bt[bt.hypos == .65].index
d = (over_65_index - center_index).values[0]

# finally calculate the cdf between the indices we got
lower_bound = center_index.values[0] - d
upper_bound = center_index.values[0] + d
limits = (
    (bt.index >= lower_bound) &
    (bt.index <= upper_bound)
)
cum_prob = bt[limits].posterior.sum().round(2) * 100
lower_prob = bt.at[lower_bound, 'hypos']
upper_prob = bt.at[upper_bound, 'hypos']
print(
    f'The range that goes from {lower_prob} '
    f'to {upper_prob:2.2} accounts for the {int(cum_prob)}% of the probability')

### Conclusion
Although the outcome is accurate to some extent this approach is not completely right as it creates a prior out of the responses and then updates it through the likelihood of the flip of the coin. Therefore, is creating extra information (since it's updating twice) that makes the credible interval narrower making it somewhat misleading.

## Solution approach

In [ ]:
# Perform local imports
import os  
import sys  
nb_dir = os.path.split(os.getcwd())[0]  
if nb_dir not in sys.path:  
    sys.path.append(nb_dir)

import numpy as np
from empiricaldist import Pmf
from soln.utils import decorate

# I'll use a uniform distribution again, although there might
# be background information we could use to choose a more
# specific prior.
hypos = np.linspace(0, 1, 101)
prior = Pmf(1, hypos)

# If the actual fraction of cheaters is `x`, the number of
# YESes is (0.5 + x/2), and the number of NOs is (1-x)/2
likelihood = {
    'Y': 0.5 + hypos/2,
    'N': (1-hypos)/2
}

dataset = 'Y' * 80 + 'N' * 20

posterior = prior.copy()

for data in dataset:
    posterior *= likelihood[data]

posterior.normalize()

posterior.plot(label='80 YES, 20 NO')
decorate(xlabel='Proportion of cheaters',
         ylabel='PMF')